In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [ ]:
#world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

world = gpd.read_file(url)

world.plot()
plt.show()

In [ ]:
m = folium.Map(location = [0,0], zoom_start=2)
m 


In [ ]:
def create_map(location, zoom_start=10):
    map_object = folium.Map(location=location, zoom_start=zoom_start)
    return map_object

usa_map = create_map(location=[37.0902,-95.7129], zoom_start=4)
usa_map

In [ ]:
boulder_map = create_map(location=[40.015,-105.27], zoom_start=12)
boulder_map

In [ ]:
usa_map.save('usa_map.html')
boulder_map.save('boulder.map.html')

In [ ]:
from geopy.distance import geodesic

In [ ]:
data = {
    'City': ['Denver','Austin'],
    'Latitude': [39.7392, 30.2672],
    'Longitude': [-104.9903,-97.7431]
}

In [ ]:
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['Longitude'],data['Latitude']))
coords_denver = (gdf.loc[0,'Latitude'],gdf.loc[0,'Longitude'])
coords_austin = (gdf.loc[1,'Latitude'],gdf.loc[1,'Longitude'])

distance = geodesic(coords_denver, coords_austin).kilometers
print(f"distance: {distance}")

In [ ]:
lat = gdf['Latitude'].mean()
lon = gdf['Longitude'].mean()

In [ ]:
midpoint = [lat,lon]
m = folium.Map(location=midpoint, zoom_start=4)

for idx, row in gdf.iterrows():
    folium.Marker(
        location = [row['Latitude'],row['Longitude']],popuu=row['City']
    ).add_to(m)

line = folium.PolyLine(locations=[coords_denver, coords_austin],
                        color='blue',
                        weight=2.5,
                        opacity=1)

m.add_child(line)
m

In [ ]:
import folium as fl
from streamlit_folium import st_folium
import streamlit as st

In [ ]:

def get_pos(lat,lng):
    return lat,lng

m = fl.Map()

m.add_child(fl.LatLngPopup())

map = st_folium(m, height=350, width=700)


data = get_pos(map['last_clicked']['lat'],map['last_clicked']['lng'])

if data is not None:
    st.write(data)

m

In [ ]:
m = folium.Map()
m.add_child(folium.LatLngPopup())
st_folium(m)
m

In [ ]:
import folium

map_4 = folium.Map(location=[46.8527, -121.7649],
                   zoom_start=13)
folium.Marker(location=[46.8354, -121.7325], popup='Camp Muir').add_to(map_4)
map_4.click_for_marker(popup='Waypoint')
#map_4.create_map(path='mtrainier.html')
m

In [ ]:
map_3 = folium.Map(location=[46.1991, -122.1889],
                   zoom_start=13)
folium.LatLngPopup().add_to(map_3)
map_3

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="geo_coder")
address = "1600 Pennsylvania Avenue NW, Washington, DC 20500, USA"

try:
    location = geolocator.geocode(address)
    if location:
        print(f'Address: {address}')
        print(f'Latitude: {location.latitude}, Longitude: {location.longitude}')
except Exception as e:
    print(f'Error: {e}')

In [ ]:
location = geolocator.geocode(address)
print(location)

print(f'Address: {address}')
print(f'Latitude: {location.latitude}, Longitude: {location.longitude}')

In [ ]:
from ipyleaflet import Map, Marker, CircleMarker, basemaps, LayersControl


In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

In [ ]:
def geocode_city(city_name):
    geolocator = Nominatim(user_agent="geo_analysis")
    location = geolocator.geocode(city_name)
    return(location.latitude, location.longitude)

def cal_distance(point1, point2):
    return great_circle(point1, point2).kilometers



In [ ]:
if __name__ == "__main__":
    city_name = input("enter city name to find nearby big cities:")
    city_coords = geocode_city(city_name)
    print(f"Coordiantes of {city_name}: {city_coords}")
    big_cities = {
        "New York": (40.7128, -74.0060),
        "Los Angeles": (38.0522, -118.2437),
        "Chicago": (41.8781, -87.6298)
    }


nearby_cities = {}
for city, coords in big_cities.times():
    distance = cal_distance(city_coords, coords)
    if distance <= 500:
        nearby_cities[city] = coords

m = Map(center=(city_coords[0], city_coords[1]), zoom=8, basemaps= basemaps.OpenStreetMap.Mapnik)
input_city_marker = Marker(location=(city_coords[0],city_coords[1]), title=city_name)
m.add_layer(input_city_marker)
for city, coords in nearby_cities.items():
    marker = CircleMarker(location=(city_coords[0],city_coords[1]),radius=7, color = "blue", fill_color = "blue", fill_opacity=0.6)
    m.add_layer(marker)
m.add_control(LayersControl())
display(m)

US Population Density Project Below

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm
#from google.colab import files

In [ ]:
pop_data = pd.read_csv('usa_population_2019.csv')
pop_data.head(10)

In [ ]:
gdf = gpd.read_file('us-states.json')

In [ ]:
merged_data = gdf.merge(pop_data, left_on="name", right_on="Geographic Area")
merged_data.head(10)


In [ ]:
merged_data["area_km2"] = merged_data["geometry"].to_crs(epsg=3395).area /10**6
merged_data["population_density"] = merged_data["Total Resident Population"]/merged_data["area_km2"]
print(merged_data[['name','Total Resident Population','area_km2','population_density']])

In [ ]:
min_density = merged_data['population_density'].min()
max_density = merged_data['population_density'].max()

colormap = cm.linear.YlOrRd_09.scale(min_density, max_density)

In [ ]:
def style_func(feature):
    density = feature['properties']['population_density']
    return {
        'fillColor': colormap(density),
        'color':'black',
        'weight':1,
        'fillOpacity': 0.6,
    }

m = folium.Map(location=[37.8,-96], zoom_start=4)
folium.GeoJson(
    merged_data,
    style_function=style_func,
    tooltip=GeoJsonTooltip(fields=['name','population_density'],
                            aliases=['State:','Population Density:'],
                            localize=True)
).add_to(m)

colormap.caption = "Population Density"
colormap.add_to(m)
m.save('us_popdensity.html')
m 

Heatmap: Air Quality Below

In [ ]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import pandas as pd
import numpy as np 


In [ ]:
data = pd.read_csv("ad_viz_plotval_data.csv")
data.head(10)

In [ ]:
data["Date"] = pd.to_datetime(data["Date"])
data["Week"] = data["Date"].dt.isocalendar().week 

In [ ]:
weekly_mean_aqi = data.groupby(["Local Site Name","Week"]).agg({
    'Site Latitude': 'first',
    'Site Longitude': 'first',
    'Daily AQI Value': lambda x: np.mean(x)
}).reset_index()

print('Weekly Mean AQI Value per Location:')
for location in weekly_mean_aqi["Local Site Name"].unique():
    location_data = weekly_mean_aqi[weekly_mean_aqi['Local Site Name'] == location]
    print(f'Location:{location}')
    for index, row in location_data.iterrows():
        print(f'Week: {row["Week"]}, Average AQI Wale: {row["Daily AQI Value"]:.2f}')
    print('\n')


In [ ]:


df_week_list = []

for week in range(1,10):
    weekly_data = weekly_mean_aqi[weekly_mean_aqi["Week"]==week][["Site Latitude", "Site Longitude","Daily AQI Value"]]
    df_week_list.append(weekly_data.values.tolist())

basemap = folium.Map(location=[34.0549,-118.24], zoom_start=11)
HeatMapWithTime(df_week_list, radius=70, gradient={0.05:'blue',0.5:'green',0.75:'yellow',1.0:'red'},
                min_opacity=0.5,
                max_opacity=0.8,
                use_local_extrema=True).add_to(basemap)

# Add the title
loc = 'Weekly AQI'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc) 

basemap.get_root().html.add_child(folium.Element(title_html))
basemap.save('weekly_aqi_heatmap.html')

basemap



Calculating Risk Flood

In [ ]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

In [ ]:
data = pd.read_csv('AEGISDataset.csv')

In [ ]:
data.head(10)

In [ ]:
numeric_columns=['lat','lon','flood_heig','elevation','precipitat']
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors = 'coerce')
data_clean = data.dropna()
print("Nan after cleaning:")
print(data_clean.isnull().sum())

In [ ]:
data_clean["Normalized_Height"]= (data_clean['flood_heig']-data_clean['flood_heig'].min())/(data_clean['flood_heig'].max() - data_clean['flood_heig'].min())
data_clean['Normalized_Elev'] = (data_clean['elevation']-data_clean['elevation'].min())/(data_clean['elevation'].max() - data_clean['elevation'].min())
data_clean['Normalized_Precip'] = (data_clean['precipitat']-data_clean['precipitat'].min())/(data_clean['precipitat'].max() - data_clean['precipitat'].min())

In [ ]:
data_clean['Normalized_Elev'] = 1- data_clean["Normalized_Elev"]


In [ ]:
data_clean['Risk_Score'] = (data_clean["Normalized_Height"] * 0.4 + data_clean['Normalized_Elev'] * 0.3 + data_clean['Normalized_Precip'] * 0.3)

In [ ]:
print('\Risk Score for Each LOcation:')
for index, row in data_clean.iterrows():
    print(f'Latitude:{row["lat"]}, Longitude:{row["lon"]}, Risk Score:{row["Risk_Score"]}')

In [ ]:
def get_color(risk_score):
    if risk_score < 0.2: 
        return 'green'
    elif 0.2 <= risk_score < 0.4:
        return 'yellow'
    elif 0.4 <= risk_score < 0.6:
        return 'orange'
    elif 0.6 <= risk_score < 0.8:
        return 'red'
    else:
        return 'darkred'

In [ ]:
data_clean['color'] = data_clean['Risk_Score'].apply(get_color)

manila_coords = [14.599, 120.98]

manila_map = folium.Map(location=manila_coords, zoom_start=11)
heat_data = data_clean[['lat','lon','Risk_Score']].values.tolist()
HeatMap(heat_data, radius=15, max_zoom=15).add_to(manila_map)
#for index, row, in data_clean.iterrows():
#    print((f'Latitude:{row["lat"]}, Longitude:{row["lon"]}, Risk Score:{row["Risk_Score"]}, Risk Color:{row["color"]}'))

manila_map.save('flood_risk_heatmap.html')
manila_map

HeatMap Snow Cover Project

In [ ]:
import pandas as pd 
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim

In [ ]:
ny_data = pd.read_csv('nyc_Jan_Jun_2016_weat.csv')

In [ ]:
ny_data.head(10)

In [ ]:
ny_data["SNOW"] = pd.to_numeric(ny_data["SNOW"], errors='coerce')

In [ ]:
snow_pos = ny_data[ny_data["SNOW"] > 0]

In [ ]:
max_snow_per_loc = snow_pos.loc[snow_pos.groupby(['LATITUDE','LONGITUDE'])['SNOW'].idxmax()]
print("\nSnow Depth in Inches:")
for index,row in max_snow_per_loc.iterrows():
    print(f'Latitude:{row["LATITUDE"]},Longitude:{row["LONGITUDE"]}, Snow Depth:{row["SNOW"]:.2f}')

geolocator = Nominatim(user_agent="snow_depth_locator")
top_3 = max_snow_per_loc.nlargest(3,"SNOW")
print("\n Top 3 Places")
for index, row in top_3.iterrows():
    location = geolocator.reverse((row['LATITUDE'], row["LONGITUDE"]),exactly_one=True)
    address = location.address if location else "Address not found"
    print(f'Latitude:{row["LATITUDE"]},Longitude:{row["LONGITUDE"]}, Address: {address}, Snow Depth:{row["SNOW"]:.2f}')

In [ ]:
min_snow = ny_data["SNOW"].min()
print(min_snow)

In [ ]:
heat_data = max_snow_per_loc[['LATITUDE','LONGITUDE','SNOW']].values.tolist()
map_center = [40.71,-74.006]
map_nyc = folium.Map(location=map_center, zoom_start=11)
HeatMap(heat_data, radius=45).add_to(map_nyc)

map_nyc

Open street stuff

In [ ]:
import folium
import osmnx as ox 
import networkx as nx 

In [ ]:
origin = [40.73, -74.99]
dest =   [40.81, -73.96]
rd = ox.graph_from_place('Manhattan, New York, USA', network_type='drive')
origin_node = ox.distance.nearest_nodes(rd, origin[1], origin[0])
dest_node = ox.distance.nearest_nodes(rd, dest[1], dest[0])
shortest_path = nx.shortest_paths(rd, origin_node, dest_node, weight='length')
path_coords = [(rd.nodes[node]['y'],rd.nodes[node]['x']) for node in shortest_path]